<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/Robotics/Subsumption.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install aitk --upgrade --quiet

# Subsumption Architecture

This notebook defines several Python classes that allow you to simulate the Subsumption Architecture, a robot control framework first described by Rodney Brooks.

The Sumbumption Architecture is based on a hierarchical set of layers. Each layer focuses on the relevant sensors for its task and commands the actuators needed to achieve its goal. Each layer competes to control the robot at any given time step, and the one with the highest priority is given control, subsuming control from all lower layers. This framework is known for its real-time responsiveness, and can quickly adapt to changing conditions in the environment.

In this notebook we will use Subsumption Architecture to control a robot with both light and range sensors, to wander its environment to find and approach a light source.

In [2]:
from aitk.robots import *

In [3]:
import random

## Defining the Behavior class

Each layer of the architecture is known as a *behavior*.

In [4]:
class SubsumptionBehavior(object):
    """
    Each subsumption behavior maintains it's own desired actuator values.
    Each behavior also maintains a flag. When this flag is True,
    the behavior wants to respond to the current situation. However, only
    the actuator values of the highest priority behavior will control the
    robot.

    The move method updates translate, rotate, and flag.
    The controller's add method updates robot and controller.
    """
    def __init__(self):
        self.translate = 0
        self.rotate = 0
        self.flag = False
        self.robot = None
        self.controller = None

    def set_robot(self, robot):
        self.robot = robot

    def set_controller(self, controller):
        self.controller = controller

    def move(self, translate, rotate):
        self.flag = True
        self.translate = translate
        self.rotate = rotate

    def update(self):
        """
        Each behavior overrides this method to check whether it wants to control
        the robot in the current situation.
        """
        pass

## Defining the Subsumption Controller

A controller is a collection of behaviors which must be added from lowest to highest priority.

In [5]:
class SubsumptionController(object):
    """
    A SubsumptionBrain maintains a list of behaviors from lowest to
    highest priority.  On each time step, it determines the highest
    level behavior that wants to respond, and allows it to control the
    robot.
    """
    def __init__(self, robot):
        self.behaviors = []
        self.robot = robot
        self.prev_behavior = None
        self.robot.state["timer"] = 0
        self.done = False

    def reset(self):
        """
        Reset key variables prior to each new run
        """
        self.prev_behavior = None
        self.robot.state["timer"] = 0
        self.done = False

    def add(self, behavior):
        """
        Adds a behavior to the list of behaviors.
        """
        behavior.set_robot(self.robot)
        behavior.set_controller(self)
        self.behaviors.append(behavior)

    def step(self):
        """
        This method is called on every time step and determines the
        robot's action.
        """
        b = self.update_all()
        # check whether any of the behaviors have achieved the goal
        if self.done:
            self.robot.speak("Goal achieved!")
            return True
        curr_behavior = self.behaviors[b].__class__.__name__
        if curr_behavior != self.prev_behavior:
            self.robot.speak(curr_behavior)
            #print(curr_behavior, end=" ")
            self.prev_behavior = curr_behavior
        self.robot.move(self.behaviors[b].translate,self.behaviors[b].rotate)

    def update_all(self):
        """
        Returns the index of the highest priority behavior that wants
        to control the robot in the current situation.
        """
        # for all except lowest
        for b in range(len(self.behaviors) - 1, 0, -1):
            self.behaviors[b].flag = False
            self.behaviors[b].update()
            # if it fired, return number
            if self.behaviors[b].flag:
                return b
        # if none of the higher priority behaviors fired, return lowest
        self.behaviors[0].update()
        return 0


## Example: Find light

Here we implement several behaviors that when used together allow a robot to find a bulb in various worlds where the light is hidden behind walls.

### Wander behavior

The Wander behavior is the lowest priority behavior and will run by default when no other behavior is relevant. It simply moves forward with a small random rotation.

In [6]:
class Wander(SubsumptionBehavior):
    """
    Moves forward and randomly chooses a small rotation amount.
    """
    def update(self):
        self.move(.75, random.random() - 0.5)


### SeekLight behavior

The SeekLight behavior checks both its left and right light sensors, and if they are sensing some light it moves the robot in the direction where it is sensing more light.

In [7]:
class SeekLight(SubsumptionBehavior):
    """
    When light is sensed, moves forward slowly while turning towards
    the side which is sensing more light.
    """
    def update(self):
        left_light = self.robot["left-light"].get_brightness()
        right_light = self.robot["right-light"].get_brightness()
        diff_light = left_light - right_light
        if  max(left_light, right_light) > 0.0 and abs(diff_light) > 0.02:
            if diff_light < 0:
                # light stronger to right
                self.move(0.2, -0.1)
            else:
                # light stronger to left
                self.move(0.2, 0.1)

### Avoid behavior

The Avoid behavior checks the range sensors to determine the robot is too close to any obstacles, and moves the robot away from the closest obstacle.

In [8]:
class Avoid(SubsumptionBehavior):
    """
    Turns away from obstacles. Uses a timer to repeatedly turn in the same direction
    to alleviate dithering in corners.
    """
    def update(self):
        left_ir = self.robot["left-ir"].get_distance()
        right_ir = self.robot["right-ir"].get_distance()
        if self.robot.state["timer"] > 0 or \
            left_ir < self.robot["left-ir"].get_max() or \
            right_ir < self.robot[1].get_max():
            if self.robot.state["timer"] == 0:
                self.robot.state["timer"] += 1
                if left_ir < right_ir:
                    # obstacle closer on left, rotate right
                    self.move(0.05, -0.4)
                else:
                    # obstacle closer on right, rotate left
                    self.move(0.05, 0.4)
            elif self.robot.state["timer"] > 5:
                self.robot.state["timer"] = 0
            else:
                # otherwise continue turning in the same direction as before
                self.robot.state["timer"] += 1
                self.move(self.translate, self.rotate)

### FoundLight behavior

The FoundLight behavior checks whether the robot's light sensors are nearly maxed out, indicating that the robot has found the light and can stop.

In [9]:
class FoundLight(SubsumptionBehavior):
    """
    When the light sensed is above a certain threshold, stops the robot.
    """
    def update(self):
        left_light = self.robot["left-light"].get_brightness()
        right_light = self.robot["right-light"].get_brightness()
        total_light = left_light + right_light
        if total_light > 1.95:
            # goal achieved so tell controller to stop running
            self.move(0, 0)
            self.controller.done = True

### Creating a World and a Robot

Let's create a world with a light source that is partially enclosed within a set of blue walls. In order to discover the light source the robot will have to explore the world, avoiding obstaccles, until it happens to see some light, and can then approach the light and stop.

The robot is equipped with two light sensors and two range sensors placed on both its left and right front corners.

In [10]:
world = World(width=200,height=200)
world.add_wall("blue", 50, 50, 60, 120)
world.add_wall("blue", 50, 40, 80, 50)
world.add_wall("blue", 140, 50, 150, 120)
world.add_wall("blue", 120, 40, 150, 50)
world.add_wall("blue", 50, 110, 150, 120)
world.add_bulb("yellow", 100, 90, 0, 50)
robot = Scribbler(x=150,y=160,a=180)
robot.add_device(RangeSensor(position=(6,-6),width=57.3,max=20,name="left-ir"))
robot.add_device(RangeSensor(position=(6,6),width=57.3,max=20,name="right-ir"))
robot.add_device(LightSensor(position=(6,-6),name="left-light"))
robot.add_device(LightSensor(position=(6,6),name="right-light"))
world.add_robot(robot)

Random seed set to: 546831


In [11]:
world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

### Creating a subsumption controller

Now we can create a subsumption-style controller for our robot. We will add the behaviors in from lowest to highest priorioty: Wander, SeekLight, Avoid, and FoundLight.

Why does it make sense for the Avoid behavior to have a higher priority that the SeekLight one?

In [12]:
subsumption = SubsumptionController(world.robots[0])

In [13]:
# Uses 4 layered behaviors to find a light source, layers are added from lowest to highest priority

# Layer 0: By default robot will wander, moving forward and choosing random rotations
subsumption.add(Wander())

# Layer 1: If light is detected, then turn towards it
subsumption.add(SeekLight())

# Layer 2: When an obstacle is encountered, turn away from it
subsumption.add(Avoid())

# Layer 3: When the light readings are high enough, stop
subsumption.add(FoundLight())

In [14]:
def controller(robot):
    return subsumption.step()

### Testing the Subsumption controller

Feel free to rerun the code cell below as many times as you'd like. It will choose a random position for the robot in the world, reset the controller, and then allow the robot to try to find the light for at most two minutes of real time. 

NOTE: To watch the robot move, scroll back up to the world view.

Occasionally the random pose might place the robot on top of or right next to a wall and the robot will get stuck. Simply rerun it again to get a different random pose.

In [16]:
world.reset()
world.robots[0].set_random_pose()
subsumption.robot.set_max_trace_length(120)
subsumption.reset()
world.seconds(120, [controller], real_time=True)

  0%|          | 0/1200 [00:00<?, ?it/s]

Simulation stopped at: 00:00:19.70; speed 0.98 x real time


True